# Neural networks handcrafted

by Dominik Krzemiński
(26/11/18)

In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

**Goal**: teach simple network XOR

![](img/nn1.png)

# Vanilla Neural Network

Below we create numerical representation of this simple problem:

In [ ]:
X = [(0,0), (1,0), (0,1), (1,1)]
Y = [0, 1, 1, 0]

We will use the following architecture:
![](img/nn2.png)
which gives a number of parameters:
![](img/nn3.png)

Let's define our parameter vector according to this. Additionaly, I define $\eta$ learning rate. We will play around this later.

In [ ]:
eta = 0.1

In [ ]:
w1 = 2 * np.random.random(size=(2,3)) - 1
w2 = 2 * np.random.random(size=(3,1)) - 1
b1 = 2 * np.random.random(size=(1,3)) - 1
b2 = 2 * np.random.random(size=(1,1)) - 1

Typical neuron performs the operation of sum of weighted input and thresholding the activation $a$.

![](img/nn4.png)

Our activation funtion is sigmoid (you can learn more here: https://en.wikipedia.org/wiki/Sigmoid_function):

![](img/nn5.png)

Let's compute the first derivative as we might need it later.

![](img/nn6.png)

Yeah, that's a good moment to appreciate simplicity of the above formula. That's one of the reasons why ML researchers like this function.

In [ ]:
def sigmoid(x):
    return 1./(1 + np.exp(-x))

def dsigmoid(y):
    return y*(1-y)

In [ ]:
x = np.array(X[0])
y = np.array(Y[0])

We perform forward pass first:

![](img/nn7.png)

In [ ]:
act1 = np.dot(w1.T, x) + b1
lay1 = sigmoid(act1)

In [ ]:
act2 = np.dot(w2.T, lay1.T) + b2
lay2 = sigmoid(act2)

Since we have reached the last layer, we can compute the error of our prediction.
![](img/nn9.png)

In [ ]:
E = 0.5 * (y - lay2)**2

Now, we propagate this error backwards, starting from the last layer. We need to use the chain rule to get a formula for the update.

For weights it looks like this:

![](img/nn10.png)

and for bias like this:

![](img/nn11.png)

In [ ]:
delta_l2 = (y-lay2) * dsigmoid(act2)
corr_w2 = delta_l2 * lay1
corr_b2 = delta_l2 * 1

Finally, we can update our weights using the gradient descent. Note, how every word makes sense here.
- gradient $\frac{d E}{d w}$
- descent "-"

![](img/nn8.png)

In [ ]:
w2 = w2 - eta * corr_w2.T
b2 = b2 - eta * corr_b2

Layer 1 update is a little bit more complicated, as we have to use the chain rule twice:

![](img/nn12.png)

Try making it by yourself for bias.

![](img/nn13.png)


In [ ]:
delta_l1 = np.dot(w2, delta_l2) * dsigmoid(act1).T
corr_w1 = np.outer(x, delta_l1)
corr_b1 = delta_l1 * 1

We update the parameters, similarly as above.

In [ ]:
w1 = w1 - eta * corr_w1
b1 = b1 - eta * corr_b1.T

Finally, we can put this together:

In [ ]:
# Number of epochs
# ... or in other words:
# how many times we show our network all examples
N = 10000 

error = np.zeros((N,1))

for n in range(N):
    for i in range(len(X)): # iterate over all examples
        x = np.array(X[i])
        y = np.array(Y[i])
        # Forward pass, 1st layer
        act1 = np.dot(w1.T, x) + b1
        lay1 = sigmoid(act1)
        # Forward pass, 2nd layer
        act2 = np.dot(w2.T, lay1.T) + b2
        lay2 = sigmoid(act2)
        # Computing error
        E = 0.5*(lay2 - y)**2
        error[n] += E[0]
        # Backprop, 2nd layer
        delta_l2 = (lay2-y) * dsigmoid(lay2)
        corr_w2 = (delta_l2 * lay1).T
        corr_b2 = delta_l2 * 1
        # Backprop, 1st layer
        delta_l1 = np.dot(w2, delta_l2) * dsigmoid(lay1).T
        corr_w1 = np.outer(x, delta_l1)
        corr_b1 = (delta_l1 * 1).T
        w2 = w2 - eta * corr_w2
        b2 = b2 - eta * corr_b2
        w1 = w1 - eta * corr_w1
        b1 = b1 - eta * corr_b1
        if n % 1000 == 0:
            print('| {}, {:.3f}'.format(y, lay2[0][0]), end='')
    if n % 1000 == 0:
        print(' <', '-' * 3, n)
    error[n] /= len(X)

In [ ]:
plt.plot(np.arange(N), error)
plt.ylabel('error')
plt.xlabel('epoch')

Above we implemented so called stochastic gradient descent. We update weights right after seeing a new example.

Below we slightly change our code to follow batch gradient descent schema. We update parameters after seeing a batch of examples. In  our case all four.

In [ ]:
w1 = 2 * np.random.random(size=(2,3)) - 1
w2 = 2 * np.random.random(size=(3,1)) - 1
b1 = 2 * np.random.random(size=(1,3)) - 1
b2 = 2 * np.random.random(size=(1,1)) - 1

In [ ]:
N = 10000
error = np.zeros((N,1))
for n in range(N):
    Dw_1 = np.zeros((2,3))
    Dw_2 = np.zeros((3,1))
    Db_1 = np.zeros((1,3))
    Db_2 = np.zeros((1,1))

    for i in range(len(X)): # iterate over all examples
        x = np.array(X[i])
        y = np.array(Y[i])
        # Forward pass, 1st layer
        act1 = np.dot(w1.T, x) + b1
        lay1 = sigmoid(act1)
        # Forward pass, 2nd layer
        act2 = np.dot(w2.T, lay1.T) + b2
        lay2 = sigmoid(act2)
        # Computing error
        E = 0.5*(lay2 - y)**2
        error[n] += E[0]
        # Backprop, 2nd layer
        delta_l2 = (lay2-y) * dsigmoid(lay2)
        corr_w2 = (delta_l2 * lay1).T
        corr_b2 = delta_l2 * 1
        # Backprop, 1st layer
        delta_l1 = np.dot(w2, delta_l2) * dsigmoid(lay1).T
        corr_w1 = np.outer(x, delta_l1)
        corr_b1 = (delta_l1 * 1).T
        Dw_2 += corr_w2
        Dw_1 += corr_w1
        Db_2 += corr_b2
        Db_1 += corr_b1
        if n % 1000 == 0:
            print('| {}, {:.3f}'.format(y, lay2[0][0]), end='')
    if n % 1000 == 0:
        print(' <', '-' * 3, n)
    w2 = w2 - eta * Dw_2
    b2 = b2 - eta * Db_2
    w1 = w1 - eta * Dw_1
    b1 = b1 - eta * Db_1
    error[n] /= len(X)

In [ ]:
plt.plot(np.arange(N), error)
plt.ylabel('error')
plt.xlabel('epoch')

Is there anything you can notice by comparing these two graphs?

# Vanilla Recurrent Neural Network

Some people think that RNN are more difficult to understand. Well... conceptually there are some nuances, but implementation is not that difficult.

First of all, we need to change the way we think about our problem. Now our $X_r$ is not simply a pair of numbers, but a sequence. We will try to teach our network slightly different problem. To recognise that 1 should follow only two preceding zeros.

```
0 -> 0 -> 1
1 -> 0 -> 0
0 -> 1 -> 0
1 -> 1 -> 0
```

Let's try to teach our network that.

Our architecture looks like this:

![](img/nn14.png)

Quite similar, isn't it?

The only difference is the recurrent layer with parameters $w_h$ and $b_h$.

In [ ]:
w1 = 2 * np.random.random(size=(1,3)) - 1
w2 = 2 * np.random.random(size=(3,1)) - 1
b1 = 2 * np.random.random(size=(1,3)) - 1
b2 = 2 * np.random.random(size=(1,1)) - 1

In [ ]:
wh = 2 * np.random.random(size=(3,3)) - 1
bh = 2 * np.random.random(size=(1,3)) - 1

Since the architecture is so similar, it's easy to guess that most of the equations stay the same.

Obviously, there is a difference in layer 1:

![](img/nn15.png)

The update rules for $w_1$, $w_2$, $b_1$ and $b_2$ look exactly the same. We need to add update rule for the recurrent hidden layer.

![](img/nn16.png)

In [ ]:
Xr = [[0,0,1], [1,1,0], [1,0,0], [0,1,0]]

The main difference in adjusting the previous code lays in the time-step propagation. Notice variables `lay1_mem` and `lay2_mem` that contain previous values of layers 1 and 2 respectively (`mem` stands for memory). We need to "remember" previous values in order to propagate the error backwards in time. I hope that it agrees with intuitive behaviour of recurrent neural networks.

In [ ]:
N = 10000
error = np.zeros((N,1))

corr_w1 = np.zeros((1,3))
corr_b1 = np.zeros((1,3))
corr_w2 = np.zeros((3,1))
corr_b2 = np.zeros((1,1))
corr_wh = np.zeros((3,3))
corr_bh = np.zeros((1,3))

for n in range(N):
    for i in range(len(Xr)):
        lay1_mem = [np.zeros((1,3))]  # ! NEW
        lay2_mem = []                  # ! NEW
        for k in range(2):             # ! NEW
            x = np.array(Xr[i][k])
            y = np.array(Xr[i][k + 1])
            # Forward pass, 1st layer
            act1 = np.dot(w1.T, x) + b1.T + (np.dot(lay1_mem[-1], wh) + bh).T # ! NEW
            lay1 = sigmoid(act1).T
            # Forward pass, 2nd layer
            act2 = np.dot(w2.T, lay1.T) + b2
            lay2 = sigmoid(act2)
            # Computing error
            E = 0.5*(lay2 - y)**2
            error[n] += E[0]
            lay1_mem.append(np.copy(lay1))      # ! NEW
            lay2_mem.append(np.copy(lay2))       # ! NEW
            if n % 1000 == 0:
                print('{} - > {} ({:.3f}|{:.0f})'.format(x, y, lay2[0][0], 1.*(lay2[0][0]>0.5)), end='')
        for k in range(1,-1,-1):                 # ! NEW
            x = np.array(Xr[i][k])
            y = np.array(Xr[i][k + 1])
            # Backprop, 2nd layer
            delta_l2 = (lay2_mem[k]-y) * dsigmoid(lay2_mem[k])
            corr_w2 += (delta_l2 * lay1_mem[k+1]).T
            corr_b2 += delta_l2 * 1
            # Backprop, 1st layer
            delta_l1 = np.dot(w2, delta_l2) * dsigmoid(lay1_mem[k+1]).T
            corr_w1 += np.outer(x, delta_l1)
            corr_b1 += (delta_l1 * 1).T
            # Backprob, recurrent layer
            corr_wh += np.outer(lay1_mem[k], delta_l1) # ! NEW
            corr_bh += (delta_l1 * 1).T                 # ! NEW
        w2 = w2 - eta * corr_w2
        b2 = b2 - eta * corr_b2
        w1 = w1 - eta * corr_w1
        b1 = b1 - eta * corr_b1
        wh = wh - eta * corr_wh # ! NEW
        bh = bh - eta * corr_bh # ! NEW
        corr_w2 *= 0
        corr_b2 *= 0
        corr_w1 *= 0
        corr_b1 *= 0
        corr_wh *= 0
        corr_bh *= 0
        if n % 1000 == 0:
            print(' <', '-' * 3, n)
    error[n] /= len(X)*2

In [ ]:
plt.plot(np.arange(N), error)
plt.ylabel('error')
plt.xlabel('epoch')